<a href="https://colab.research.google.com/github/SampMark/ETL-de-dados-da-PNP/blob/main/RFETP/GitHub_ETL_Unidades_da_Rede_Federal_de_EPCT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ETL de arquivo CSV do portal de dados abertos MEC**: "Unidades da Rede Federal de EPCT"

### **Fluxo de trabalho:**

1. Extrair CSV oficial do portal do MEC (OData Olinda MEC).

2. Tratar textos (mojibake, acentuação, Unicode), padronizar colunas e enriquecer com Cidade/UF e Regioes.

3. Carregar o DataFrame em uma aba do Google Sheets (link público).

In [1]:
# Bibliotecas necessárias
import pandas as pd
import requests
import io
from io import BytesIO
import unicodedata
import re
from typing import Optional
from pathlib import Path
print("Importação de bibliotecas concluída.")

# Autenticação no Google Sheets
from google.colab import auth
import gspread
import google.auth
from gspread import Client
print("Autenticação concluída.")

Importação de bibliotecas concluída.
Autenticação concluída.


In [2]:
# @title **ETAPA 1: Download, tratamento e exibição dos dados da RFEPT**

"""
Leitura de CSV dados abertos MEC: "Unidades da Rede Federal de EPCT"
- Faz download do arquivo direto da URL oficial
- Detecta/força codificação adequada (UTF-8/UTF-8-SIG/CP1252/Latin-1)
- Corrige possíveis casos de "mojibake" (ex.: "Ã¡, Ã£, Ã©, Ã³")
- Auto-detecta separador (muitos CSVs do MEC usam ';')
- Exibe dimensões (n_linhas x n_colunas) e uma prévia do DataFrame

Uso sugerido (Jupyter/Colab):
    %run -i mec_unidades_rede_federal_csv_leitura_robusta.py

Ou importe a função principal e chame diretamente:
    from mec_unidades_rede_federal_csv_leitura_robusta import load_mec_unidades
    df = load_mec_unidades(show_preview=True)
"""

# ----------------------------
#  Download, tratamento e exibição dos dados dos IFs e Campus
# ----------------------------

# Download do Portal de Dados Abertos do MEC
# https://dados.gov.br/dados/conjuntos-dados/rede-federal-de-educacao-profissional-cientifica-e-tecnologica
URL = (
    "https://olinda.mec.gov.br/olinda-ide/servico/PDA_SETEC/versao/v1/odata/"  # Link específico
    "RedeFederal_Lista_Instituicoes_RFEPCT_2022?$format=text/csv"
)

# Cria o diretório de saída dentro do ambiente Colab
OUT_DIR = Path.cwd() / "dados_mec"
OUT_DIR.mkdir(exist_ok=True)
CSV_PATH = OUT_DIR / "Unidades_Rede_Federal_EPCT.csv"

# Mapa de renomeação para Campus_IF
MAPA_CORRECAO_CAMPUS = {
    r'^Unidade$': 'Uned',
    'Campus Avançado Abelardo Luz': 'Campus Abelardo Luz',
    'Campus Avançado Arcos': 'Campus Arcos',
    'Campus Avançado Astorga': 'Campus Astorga',
    'Campus Avançado Bom Sucesso': 'Campus Bom Sucesso',
    'Campus Avançado Bonfim': 'Campus Bónfim',
    'Campus Avançado Cabedelo Centro': 'Campus Cabedelo Centro',
    'Campus Avançado Carolina': 'Campus Carolina',
    'Campus Avançado Conselheiro Lafaiete': 'Campus Conselheiro Lafaiete',
    'Campus Avançado Formoso do Araguaia': 'Campus Formoso do Araguaia',
    'Campus Avançado Lagoa da Confusão': 'Campus Lagoa da Confusão',
    'Campus Avançado Ipatinga': 'Campus Ipatinga',
    'Campus Avançado Iranduba': 'Campus Iranduba',
    'Campus Avançado Jaguarão': 'Campus Jaguarão',
    'Campus Avançado João Pessoa Mangabeira': 'Campus Avançado Mangabeira',
    'Campus Avançado João Pessoa': 'Campus Avançado João Pessoa',
    'Campus Avançado José de Freitas': 'Campus José de Freitas',
    'Campus Avançado Jundiaí': 'Campus Jundiaí',
    'Campus Avançado Maricá': 'Campus Maricá',
    'Campus Avançado Natal Zona Leste': 'Campus Natal-Zona Leste',
    'Campus Avançado Oiapoque': 'Campus Oiapoque',
    'Campus Avançado Pedras de Fogo': 'Campus Pedra de Fogo',
    'Campus Avançado Pedro Afonso': 'Campus Pedro Afonso',
    'Campus Avançado PIO IX': 'Campus Pio IX',
    'Campus Avançado Piumhi': 'Campus Piumhi',
    'Campus Avançado Ponte Nova': 'Campus Ponte Nova',
    'Campus Avançado São Miguel do Guaporé': 'Campus São Miguel do Guaporé',
    'Campus Avançado São Paulo - Pinheiros': 'Campus Pinheiros',
    'Campus Avançado São Paulo - São Miguel': 'Campus São Paulo - São Miguel',
    'Campus Avançado Sinop': 'Campus Sinop',
    'Campus Avançado Teresina Dirceu Arcoverde': 'Campus Teresina Dirceu Arcoverde',
    'Campus Avançado Uberaba Parque Tecnológico': 'Campus Avançado Uberaba Parque Tecnológico (Unidade I)',
    'Campus Avançado Veranópolis': 'Campus Veranópolis',
    'Campus Avançado Viana': 'Campus Viana',
    'Campus Avançado Vigia': 'Campus Vigia',
    'Campus Duque de Caxias CP': 'Campus Duque de Caxias',
     r'^Campus Jaraguá do Sul$': 'Campus Jaraguá do Sul Centro',
    'Campus Natal Cidade Alta': 'Campus Natal-Centro Histórico',
    'Campus Santo Antÿnio de Jesus': 'Campus Santo Antônio de Jesus',
    'Campus Santo Antÿnio de Pádua': 'Campus Santo Antônio de Pádua',
    'Campus São Carlos - IFSC': 'Campus São Carlos',
    'Uned Belo Horizonte': 'Uned Belo Horizonte (Campus Nova Suíça)',
}

# ----------------------------
# Utilidades de codificação e correção
# ----------------------------

def _has_utf8_bom(b: bytes) -> bool:
    """Verifica se os bytes possuem a Marca de Ordem de Byte (BOM) UTF-8."""
    return b.startswith(b"\xEF\xBB\xBF")


def looks_mojibaked(text: Optional[str]) -> bool:
    """
    Detecta se o texto contém padrões comuns de 'mojibake' (utf-8 decodificado como latin-1/cp1252).

    Esta versão utiliza regex compilado para maior performance em grandes volumes de dados
    e consolida a lista de padrões para fácil manutenção.

    Args:
        text: A string a ser analisada.

    Returns:
        bool: True se padrões de erro de codificação forem encontrados.
    """
    if not text:
        return False

    # Lista de padrões baseada nos artefatos comuns do UTF-8 lido como Windows-1252/Latin-1.
    # Ex: 'á' (UTF-8: C3 A1) vira 'Ã¡' (Latin-1: C3=Ã, A1=¡)
    padroes = [
        r"Ã­",  # í (muito comum)
        r"Ã",  # Ó
        r"Ã¡",  # á
        r"Ã£",  # ã
        r"Ã©",  # é
        r"Ã³",  # ó
        r"Ãº",  # ú
        r"Ã§",  # ç
        r"Ã±",  # ñ
        r"Ã¢",  # â
        r"Ãª",  # ê
        r"Ã¿",  # ¿ (ou fim de sequência)
        r"Ãõ",  # õ
        r"Ã",  # Í
        r"Ã­r", # Variação comum de fim de verbo
        r"Ãµ",  # õ

        # O caractere 'Â' (U+00C2) é uma letra válida em português (ex: "Ângulo").
        # No mojibake, ele geralmente surge do byte C2 (início de sequências UTF-8 de 2 bytes).
        # Para ser mais robusto e evitar falsos positivos com a palavra "Ângulo",
        # procuramos 'Â' seguido de espaço não quebrável (artefato comum) ou isolado de forma suspeita.
        # Aqui mantivemos genérico conforme sua lista original, mas considere refinar se tiver palavras válidas com Â.
        r"Â",
    ]

    # Cria uma expressão regular única: (Ã­|Ã|Ã¡|...)
    # A flag re.IGNORECASE não é usada aqui pois mojibake é sensível a bytes exatos (Maiúsculas importam).
    regex_mojibake = re.compile("|".join(padroes))

    # O método .search() para na primeira ocorrência, sendo muito eficiente.
    return bool(regex_mojibake.search(text))


def fix_mojibake(text: Optional[str]) -> str:
    """
    Tenta corrigir o texto revertendo a decodificação incorreta.
    Útil para aplicar logo após a detecção.
    """
    if not text:
        return ""

    # Se detectado, tenta consertar
    if looks_mojibaked(text):
        try:
            # O padrão de erro comum no Brasil é:
            # Bytes UTF-8 -> Decodificados como cp1252 (Windows) ou latin1 (ISO-8859-1)
            # A correção é o inverso: Codificar como cp1252 (recupera os bytes) -> Decodificar como utf-8
            return text.encode('cp1252').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            # Se a conversão falhar, retorna o texto original para não perder dados
            return text

    return text


# def _looks_mojibaked(text: str) -> bool:
#     """Heurística simples para detectar mojibake típico (UTF-8 lido como Latin-1)."""

#     # Procure explicitamente pelos padrões típicos
#     padroes = ["Ã­", "Ã", "Ã¡", "Ã£", "Ã©", "Ã³", "Ãº", "Ã§", "Ã±", "Â", "Ã­r", "Ã", "Ã¢", "Ãª", "Ã¿", "Ãµ"]
#     # return bool(re.search(r"Ã.|Â", text))
#     return any(p in text for p in padroes)


def _demojibake(s: str) -> str:
    """Tenta corrigir mojibake comum recodificando latin1 -> utf-8.
    Se falhar, retorna o original.
    """
    try:
        # A recodificação corrige os caracteres
        return s.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")
    except Exception:
        return s


def _normalize_unicode(s: str) -> str:
    """Normaliza Unicode (útil para acentuação consistente)."""
    return unicodedata.normalize("NFC", s)


def _fix_dataframe_text(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica correção de mojibake e normalização de Unicode em colunas de texto,
    incluindo a correção específica para 'ÿ'."""
    df = df.copy() # Evita SettingWithCopyWarning
    obj_cols = df.select_dtypes(include=["object"]).columns
    for col in obj_cols:
        # Garante que é string
        s = df[col].astype(str).fillna('')

        # Correção específica para 'ÿ' substituindo por 'ô' antes de verificar mojibake
        s = s.str.replace('ÿ', 'ô', regex=False)

        # Corrige mojibake se detectado
        mask = s.apply(looks_mojibaked)
        if mask.any():
            s[mask] = s[mask].map(_demojibake)

        # Normaliza Unicode
        df[col] = s.map(_normalize_unicode)

    return df

# ----------------------------
# Download e leitura
# ----------------------------

def _download_bytes(url: str, timeout: int = 60) -> bytes:
    """Faz o download dos dados brutos."""
    with requests.get(url, stream=True, timeout=timeout) as r:
        r.raise_for_status()
        return r.content


def _try_read_csv_from_bytes(data: bytes, encoding: str) -> pd.DataFrame:
    """Tenta ler o CSV a partir dos bytes, auto-detectando o separador."""
    # sep=None + engine='python' para autodetectar ',' ou ';'!
    # O arquivo do MEC usa ponto-e-vírgula ';', este método é mais robusto
    return pd.read_csv(BytesIO(data), sep=None, engine="python", encoding=encoding)


# ----------------------------
# Funções de tratamento
# ----------------------------

def remove_campus_suffixes(df: pd.Series) -> pd.Series:
    """
    Remova sufixos de institutos federais da coluna 'Campus' de forma otimizada.

    Utiliza uma única expressão regular para encontrar e remover os padrões
    apenas no final da string, garantindo eficiência e precisão.
    """
    # Cria um padrão regex que une todas as siglas dos IFs.
    # \s*-\s* -> busca pelo hífen com ou sem espaços ao redor.
    # (CPII|IF Baiano|IFMA|IFMG|IFPB|IFPR|IFRJ|IFSP|IFS|IFTO) -> grupo que captura qualquer uma das siglas listadas.
    # $ -> âncora que garante que a busca seja feita apenas no final da string.
    regex_pattern = r'\s*-\s*(CPII|IF Baiano|IFMA|IFMG|IFPB|IFPR|IFRJ|IFSP|IFS|IFTO)$'

    # Executa a substituição uma única vez para todo o DataFrame
    return df.str.replace(regex_pattern, "", regex=True).str.strip()


def create_address_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cria a coluna 'Cidade-UF' mesclando Município e UF com '/'.
    Assume que as colunas originais 'NOME_MUNICIPIO_UNIDADE_ENSINO' e 'SIGLA_UF_UNIDADE_ENSINO' existem.
    """
    df = df.copy()
    municipio_col = 'NOME_MUNICIPIO_UNIDADE_ENSINO'
    uf_col = 'SIGLA_UF_UNIDADE_ENSINO'

    # Verifica se as colunas necessárias existem
    if municipio_col in df.columns and uf_col in df.columns:
        # Converte para string e preenche NaNs com string vazia antes de combinar
        df['Endereco_Padronizado'] = df[municipio_col].astype(str).fillna('') + '/' + df[uf_col].astype(str).fillna('')
    else:
        print(f"Aviso: Colunas '{municipio_col}' ou '{uf_col}' não encontradas para criar a coluna 'Endereco_Padronizado'.")

    return df


def add_regioes_column(df: pd.DataFrame) -> pd.DataFrame:
    """Adiciona coluna 'Regioes' com base na UF brasileira."""
    mapa_regioes = {
        # Norte
        'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte',
        'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
        # Nordeste
        'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste',
        'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste',
        'RN': 'Nordeste', 'SE': 'Nordeste',
        # Centro-Oeste
        'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
        # Sudeste
        'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
        # Sul
        'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
    }

    df = df.copy()
    if 'UF' not in df.columns:
        print("Aviso: Coluna 'UF' não encontrada para criar 'Regioes'.")
        return df
    df['Regioes'] = df['UF'].astype(str).str.upper().map(mapa_regioes)
    return df


def rename_institutions(df: pd.DataFrame) -> pd.DataFrame:
    """Renomeia instituições específicas na coluna 'NOME_INSTITUICAO'."""
    df = df.copy()
    rename_map = {
        "Instituto Federal de Rondÿnia": "Instituto Federal de Rondônia",
    }
    if 'NOME_INSTITUICAO' in df.columns:
        df['NOME_INSTITUICAO'] = df['NOME_INSTITUICAO'].replace(rename_map)
    else:
        print("Aviso: Coluna 'NOME_INSTITUICAO' não encontrada para renomear instituições.")
    return df


def rename_campus_if(df: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    """
    Renomeia os valores na coluna 'NOME_UNIDADE_ENSINO' do DataFrame com base em um mapa fornecido.

    Args:
        df: O DataFrame de entrada.
        mapping: Um dicionário onde a chave é o valor atual e o valor é o novo valor.

    Returns:
        O DataFrame com a coluna 'NOME_UNIDADE_ENSINO' atualizada.
    """

    # Se o DataFrame estiver vazio ou mapping for None, retorna o original
    if df.empty or not mapping:
        return df

    # Para garantir que substituições mais específicas (e.g., strings mais longas
    # que podem conter outras strings como substring) sejam aplicadas primeiro,
    # classificamos os itens do mapa pelas chaves (padrões) em ordem decrescente de comprimento.
    # Isso ajuda a evitar que uma substituição de uma substring ocorra antes da substituição
    # da string completa desejada, quando regex=True é usado.
    sorted_mapping_items = sorted(mapping.items(), key=lambda item: len(item[0]), reverse=True)

    # Aplica as substituições em ordem.
    # Usamos .str.replace para aplicar o padrão regex a cada string na série.
    current_series = df['NOME_UNIDADE_ENSINO'].astype(str).copy() # Garante tipo string e trabalha em uma cópia

    for pattern, replacement in sorted_mapping_items:
        current_series = current_series.str.replace(pattern, replacement, regex=True)

    df['NOME_UNIDADE_ENSINO'] = current_series

    return df


def criar_uid_padronizado(texto):
    """
    Recebe uma string, remove acentos, converte para minúsculas
    e substitui caracteres não alfanuméricos por underline.
    """
    if not isinstance(texto, str):
        return str(texto) if pd.notnull(texto) else ""

    # 1. Normalização Unicode (separa o acento da letra)
    texto_normalizado = unicodedata.normalize('NFKD', texto)

    # 2. Mantém apenas caracteres ASCII (removes os acentos) e converte para minúsculas
    texto_sem_acento = texto_normalizado.encode('ASCII', 'ignore').decode('utf-8').lower()

    # 3. Substitui tudo que NÃO for letra ou número por underline (_)
    # O regex [^a-z0-9]+ busca grupos de caracteres especiais ou espaços
    texto_limpo = re.sub(r'[^a-z0-9]+', '_', texto_sem_acento)

    # 4. Remove underlines sobrando no início ou fim
    return texto_limpo.strip('_')


def load_mec_unidades(url: str = URL, save_local: bool = True, show_preview: bool = True) -> pd.DataFrame:
    """Baixa e lê o CSV do MEC, trata texto, cria colunas extras e retorna DataFrame final."""

    try:
        raw = _download_bytes(url)
    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Falha no download da URL {url}: {e}")

    # Salva os bytes crus (sem transformação)
    if save_local:
        CSV_PATH.write_bytes(raw)

    # Ordem de tentativas de codificação
    candidates = ["utf-8-sig", "utf-8", "cp1252", "latin-1"]
    if _has_utf8_bom(raw):
        # Prioriza utf-8-sig se o BOM for detectado
        candidates = ["utf-8-sig", "utf-8", "cp1252", "latin-1"]

    last_err = None
    used_encoding = None
    df = None

    for enc in candidates:
        try:
            df = _try_read_csv_from_bytes(raw, enc)
            used_encoding = enc
            break
        except (UnicodeDecodeError, pd.errors.ParserError) as e:
            last_err = e
        except Exception as e:
            last_err = e
            continue

    if df is None:
        raise RuntimeError(f"Falha ao decodificar CSV ou determinar separador. Último erro: {last_err}")

    # Correções de texto
    df = _fix_dataframe_text(df)

    # Renomeia instituições específicas ANTES de renomear a coluna 'NOME_INSTITUICAO'
    df = rename_institutions(df)

    # Aplica a função de renomeação de campus
    df = rename_campus_if(df, MAPA_CORRECAO_CAMPUS)

    # Aplica a remoção de sufixos no Campus
    if 'NOME_UNIDADE_ENSINO' in df.columns:
        df['NOME_UNIDADE_ENSINO'] = remove_campus_suffixes(df['NOME_UNIDADE_ENSINO'])

    # Cria a coluna 'Cidade-UF' ANTES de renomear as colunas originais usadas
    df = create_address_column(df)

    # 1. Mapeamento das colunas originais para os novos nomes
    rename_map = {
        'ANO': 'Ano_Criacao',
        'SIGLA_INSTITUICAO': 'Sigla_IF',
        'NOME_INSTITUICAO': 'Nome_IF',
        'NOME_UNIDADE_ENSINO': 'Campus_IF',
        'NOME_MUNICIPIO_UNIDADE_ENSINO': 'Municipio',
        'SIGLA_UF_UNIDADE_ENSINO': 'UF',
    }

    # Renomeia as colunas
    df = df.rename(columns=rename_map)

    # Cria a coluna UID padronizada
    if 'Sigla_IF' in df.columns and 'Campus_IF' in df.columns:
        df['UID'] = (df['Sigla_IF'] + '_' + df['Campus_IF']).apply(criar_uid_padronizado)
    else:
        print("Aviso: Colunas 'Sigla_IF' ou 'Campus_IF' não encontradas para criar a coluna 'UID'.")

    # Adiciona a coluna 'Regioes'
    df = add_regioes_column(df)

    # Ordena o DataFrame por 'UF' e 'Nome_IF'
    if 'UF' in df.columns and 'Nome_IF' in df.columns:
        df = df.sort_values(by=['UF', 'Nome_IF']).reset_index(drop=True)
    else:
        print("Aviso: Colunas 'UF' ou 'Nome_IF' não encontradas para ordenação.")


    # Lista as colunas a serem mantidas na ordem desejada, incluindo a nova coluna 'Cidade-UF' e 'Regioes'
    colunas_desejadas_final = [
        'UID',
        'Ano_Criacao',
        'Sigla_IF',
        'Nome_IF',
        'Campus_IF',
        'Regioes',
        'UF',
        'Municipio',
        'Endereco_Padronizado'
    ]

    # Seleciona e reordena apenas as colunas desejadas e renomeadas
    df = df[[col for col in colunas_desejadas_final if col in df.columns]]

    # Informação e prévia
    if show_preview:
        n_linhas, n_colunas = df.shape
        print("-" * 80)
        print("Dimensão da Rede Federal de Educação Profissional, Científica e Tecnológica (RFEPCT)")
        print("-" * 80)
        print(f"URL: {url}")
        print(f"Codificação utilizada: {used_encoding}")
        print(f"Dimensões: {n_linhas} linhas x {n_colunas} colunas\n")
        print("-" * 80)
        print("EXIBIR PRIMEIRAS LINHAS DE DATAFRAME `df_rede_federal`")
        display(df.head(20))
        print("-" * 80)
        print("TIPOS DE DADOS E VALORES NÃO-NULOS:")
        df.info()

    return df

if __name__ == "__main__":
    df_rede_federal = load_mec_unidades(show_preview=True)

--------------------------------------------------------------------------------
Dimensão da Rede Federal de Educação Profissional, Científica e Tecnológica (RFEPCT)
--------------------------------------------------------------------------------
URL: https://olinda.mec.gov.br/olinda-ide/servico/PDA_SETEC/versao/v1/odata/RedeFederal_Lista_Instituicoes_RFEPCT_2022?$format=text/csv
Codificação utilizada: utf-8-sig
Dimensões: 644 linhas x 9 colunas

--------------------------------------------------------------------------------
EXIBIR PRIMEIRAS LINHAS DE DATAFRAME `df_rede_federal`


,UID,Ano_Criacao,Sigla_IF,Nome_IF,Campus_IF,Regioes,UF,Municipio,Endereco_Padronizado
0,ifac_campus_rio_branco,2010,IFAC,Instituto Federal do Acre,Campus Rio Branco,Norte,AC,Rio Branco,Rio Branco/AC
1,ifac_campus_sena_madureira,2010,IFAC,Instituto Federal do Acre,Campus Sena Madureira,Norte,AC,Sena Madureira,Sena Madureira/AC
2,ifac_campus_tarauaca,2013,IFAC,Instituto Federal do Acre,Campus Tarauacá,Norte,AC,Tarauacá,Tarauacá/AC
3,ifac_campus_cruzeiro_do_sul,2010,IFAC,Instituto Federal do Acre,Campus Cruzeiro do Sul,Norte,AC,Cruzeiro do Sul,Cruzeiro do Sul/AC
4,ifac_campus_xapuri,2013,IFAC,Instituto Federal do Acre,Campus Xapuri,Norte,AC,Xapuri,Xapuri/AC
5,ifac_campus_rio_branco_baixada_do_sol,2014,IFAC,Instituto Federal do Acre,Campus Rio Branco Baixada do Sol,Norte,AC,Rio Branco,Rio Branco/AC
6,ifal_campus_penedo,2010,IFAL,Instituto Federal de Alagoas,Campus Penedo,Nordeste,AL,Penedo,Penedo/AL
7,ifal_campus_piranhas,2012,IFAL,Instituto Federal de Alagoas,Campus Piranhas,Nordeste,AL,Piranhas,Piranhas/AL
8,ifal_campus_santana_do_ipanema,2013,IFAL,Instituto Federal de Alagoas,Campus Santana do Ipanema,Nordeste,AL,Santana do Ipanema,Santana do Ipanema/AL
9,ifal_campus_sao_miguel_dos_campos,2013,IFAL,Instituto Federal de Alagoas,Campus São Miguel dos Campos,Nordeste,AL,São Miguel dos Campos,São Miguel dos Campos/AL


--------------------------------------------------------------------------------
TIPOS DE DADOS E VALORES NÃO-NULOS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   UID                   644 non-null    object
 1   Ano_Criacao           644 non-null    int64 
 2   Sigla_IF              644 non-null    object
 3   Nome_IF               644 non-null    object
 4   Campus_IF             644 non-null    object
 5   Regioes               644 non-null    object
 6   UF                    644 non-null    object
 7   Municipio             644 non-null    object
 8   Endereco_Padronizado  644 non-null    object
dtypes: int64(1), object(8)
memory usage: 45.4+ KB


In [3]:
# Com base nas 'Sigla_IF' conte o número de IF por 'UF'
contagem_ifs_por_uf = df_rede_federal.groupby('UF')['Sigla_IF'].nunique().reset_index()
contagem_ifs_por_uf.rename(columns={'Sigla_IF': 'Número_de_IFs'}, inplace=True)

# Filtrar para mostrar apenas UFs com 2 ou mais IFs
ufs_com_2_ou_mais_ifs = contagem_ifs_por_uf[contagem_ifs_por_uf['Número_de_IFs'] >= 2]

print("-" * 80)
print("Contagem de IFs por UF (apenas UFs com 2 ou mais IFs):")
print("-" * 80)
display(ufs_com_2_ou_mais_ifs)

--------------------------------------------------------------------------------
Contagem de IFs por UF (apenas UFs com 2 ou mais IFs):
--------------------------------------------------------------------------------


,UF,Número_de_IFs
4,BA,2
8,GO,2
10,MG,6
15,PE,2
18,RJ,4
22,RS,3
23,SC,2


In [4]:
# Exibir valores exclusivos das colunas 'Sigla_IF' e 'Nome_IF' e gerar um CSV
ifs_unicos = df_rede_federal[['Sigla_IF', 'Nome_IF']].drop_duplicates().reset_index(drop=True)

print("-" * 80)
print("Valores exclusivos das colunas 'Sigla_IF' e 'Nome_IF':")
print("-" * 80)
display(ifs_unicos)

# Gerar CSV
csv_path_ifs = OUT_DIR / "ifs_unicos.csv"
ifs_unicos.to_csv(csv_path_ifs, index=False)

print(f"\nArquivo CSV gerado em: {csv_path_ifs}")

--------------------------------------------------------------------------------
Valores exclusivos das colunas 'Sigla_IF' e 'Nome_IF':
--------------------------------------------------------------------------------


,Sigla_IF,Nome_IF
0,IFAC,Instituto Federal do Acre
1,IFAL,Instituto Federal de Alagoas
2,IFAM,Instituto Federal do Amazonas
3,IFAP,Instituto Federal do Amapá
4,IF Baiano,Instituto Federal Baiano
5,IFBA,Instituto Federal da Bahia
6,IFCE,Instituto Federal do Ceará
7,IFB,Instituto Federal de Brasília
8,IFES,Instituto Federal do Espírito Santo
9,IF Goiano,Instituto Federal Goiano



Arquivo CSV gerado em: /content/dados_mec/ifs_unicos.csv


In [5]:
df_rede_federal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   UID                   644 non-null    object
 1   Ano_Criacao           644 non-null    int64 
 2   Sigla_IF              644 non-null    object
 3   Nome_IF               644 non-null    object
 4   Campus_IF             644 non-null    object
 5   Regioes               644 non-null    object
 6   UF                    644 non-null    object
 7   Municipio             644 non-null    object
 8   Endereco_Padronizado  644 non-null    object
dtypes: int64(1), object(8)
memory usage: 45.4+ KB


In [6]:
# @title **ETAPA 2. Exportar DataFrame `df_rede_federal`para o Google Sheets**

# ======================== Configurações da Planilha ========================
# ID da planilha
ID_PLANILHA = '18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk' # Link público
# Nome da aba. Se não existir, será criada.
NOME_ABA = 'Mapa_RFEPCT_MEC_2022'
# ==============================================================

# 1. Autenticação do Google Colab
print("Autenticando-se no Google...")
# Inicia autenticação no Google
auth.authenticate_user()
print("Autenticação concluída.")

# 2. Inicializar o cliente gspread
# Utilizar as credenciais padrão estabelecidas pelo auth.authenticate_user()
credentials, project = google.auth.default()
gc = Client(auth=credentials)

try:
    # 3. Abrir a planilha usando o ID (Chave)
    sh = gc.open_by_key(ID_PLANILHA)
    print(f"Planilha ID: '{ID_PLANILHA}' acessada com sucesso.")

    # 4. Selecionar ou criar a aba (worksheet)
    try:
        # Tenta selecionar a aba existente
        worksheet = sh.worksheet(NOME_ABA)
        # Limpa o conteúdo existente para substituí-lo
        worksheet.clear()
        print(f"Aba '{NOME_ABA}' limpa e pronta para receber os novos dados.")
    except gspread.exceptions.WorksheetNotFound:
        # Se não encontrar, adiciona uma nova aba
        # Especificar número de linhas/colunas para garantir espaço
        worksheet = sh.add_worksheet(title=NOME_ABA, rows="1000", cols="10")
        print(f"Aba '{NOME_ABA}' criada.")

    # 5. Converter o DataFrame para uma lista de listas (formato que o gspread usa)
    # Inclui o cabeçalho (nomes das colunas)
    # Usar .tolist() (sem underscore) para converter o array NumPy
    # Certifica-se que df_rede_federal está definido e não vazio
    if 'df_rede_federal' in locals() and not df_rede_federal.empty:
        dados_para_sheet = [df_rede_federal.columns.tolist()] + df_rede_federal.values.tolist()
    else:
        raise ValueError("DataFrame 'df_rede_federal' não encontrado ou está vazio.")


    # 6. Exportar os dados (atualiza toda a faixa a partir de A1)
    # Usar argumentos nomeados para evitar o DeprecationWarning
    worksheet.update(values=dados_para_sheet, range_name='A1')

    # 7. Exibir o link
    print("-" * 50)
    print("Exportação concluída com sucesso!")
    print(f"Acesse a planilha aqui: {sh.url}")
    print("-" * 50)

except gspread.exceptions.SpreadsheetNotFound:
    print("-" * 50)
    print(f"ERRO: Planilha com ID '{ID_PLANILHA}' não encontrada.")
    print("Verifique se o ID está correto ou se você possui acesso a ela.")
    print("-" * 50)
except gspread.exceptions.APIError as e:
    print("-" * 50)
    print("ERRO DE API (Permissão):")
    print("Verifique se a conta Google usada na autenticação tem permissão de 'Editor' na planilha.")
    print(f"Detalhes do erro: {e}")
    print("-" * 50)
except Exception as e:
    # Captura outros erros, incluindo problemas de permissão
    print(f"Ocorreu um erro inesperado durante a exportação: {e}")

Autenticando-se no Google...
Autenticação concluída.
Planilha ID: '18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk' acessada com sucesso.
Aba 'Mapa_RFEPCT_MEC_2022' criada.
--------------------------------------------------
Exportação concluída com sucesso!
Acesse a planilha aqui: https://docs.google.com/spreadsheets/d/18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk
--------------------------------------------------
